# Analytical Correctness

In [26]:
import json
import re
import math


# ============================================================
# 1. Extract numbers from text
# ============================================================

def extract_numbers(text):
    """
    Extract all numeric values from a given text, including:
    - Integers: 123
    - Decimals: 123.45
    - Percentages: 12.3%

    Returns normalized numeric values (percentages have '%' removed).
    """
    if not isinstance(text, str):
        return []

    pattern = r"\b\d+(?:\.\d+)?%?"
    raw_numbers = re.findall(pattern, text)

    normalized = []
    for n in raw_numbers:
        if n.endswith("%"):
            normalized.append(float(n[:-1]))
        else:
            try:
                normalized.append(float(n))
            except:
                pass
    return normalized


# ============================================================
# 2. Enhanced numeric consistency checker
# ============================================================

def check_number_consistency(numbers, processed_text, rel_tol: float = 0.02, abs_tol: float = 0.0):
    """
    Enhanced number consistency check.

    Features:
    1) Exact string match
    2) Numeric fuzzy match using math.isclose()
    3) Year range match (e.g., "2020-2024" covers 2020, 2021, 2022 …)

    Parameters:
        numbers: list of numeric values extracted from narrative
        processed_text: analysis summary (string or dict)
        rel_tol: relative tolerance for numeric fuzzy matching
        abs_tol: absolute tolerance (for small numbers)

    Returns:
        found: numbers considered covered
        missing: numbers not covered
    """
    found = []
    missing = []

    # ---- Convert processed_text to string ----
    if not isinstance(processed_text, str):
        processed_text_str = json.dumps(processed_text, ensure_ascii=False)
    else:
        processed_text_str = processed_text

    # ---- Extract numbers from processed_text for fuzzy match ----
    processed_numbers = extract_numbers(processed_text_str)

    # ---- Extract year ranges like "2020-2024" / "2020–2024" ----
    year_ranges = []
    for m in re.finditer(r"(\d{4})\s*[–-]\s*(\d{4})", processed_text_str):
        start = int(m.group(1))
        end = int(m.group(2))
        if start <= end:
            year_ranges.append((start, end))

    # ---- Check each narrative number ----
    for num in numbers:
        num_float = float(num)

        # Convert to string for exact matching
        if "." in f"{num_float}":
            num_str = f"{num_float}".rstrip("0").rstrip(".")
        else:
            num_str = str(int(num_float))

        matched = False

        # -------------------------
        # 1) Exact string match
        # -------------------------
        if num_str in processed_text_str:
            matched = True

        # -------------------------
        # 2) Numeric fuzzy match
        # -------------------------
        if not matched:
            for p in processed_numbers:
                if math.isclose(num_float, p, rel_tol=rel_tol, abs_tol=abs_tol):
                    matched = True
                    break

        # -------------------------
        # 3) Year range match
        # -------------------------
        if not matched and year_ranges:
            if num_float.is_integer():
                y = int(num_float)
                if 1900 <= y <= 2100:  # consider only valid year-like numbers
                    for start, end in year_ranges:
                        if start <= y <= end:
                            matched = True
                            break

        # -------------------------
        # Collect result
        # -------------------------
        if matched:
            found.append(num)
        else:
            missing.append(num)

    return found, missing


# ============================================================
# 3. Main evaluation pipeline
# ============================================================

# Load narratives
with open('../reports/temp_report_data.json', 'r', encoding='utf-8') as f:
    report_data = json.load(f)

# Load analysis summaries
with open('../reports/analysis_summary_data.json', 'r', encoding='utf-8') as f:
    analysis_summary_data = json.load(f)

# Safety check
if len(report_data) != len(analysis_summary_data):
    print("WARNING: report_data and analysis_summary_data have different lengths!")
    print(f"len(report_data) = {len(report_data)}, len(analysis_summary_data) = {len(analysis_summary_data)}")

evaluation_results = []

for idx, (item, summary_item) in enumerate(zip(report_data, analysis_summary_data)):
    section_title = item.get("section_title", "")
    subsection_title = item.get("subsection_title", "")

    narrative = item.get("narrative", "")
    processed = summary_item.get("analysis summary", "")

    numbers = extract_numbers(narrative)

    if not numbers:
        evaluation_results.append({
            "section_index": idx + 1,
            "section_title": section_title,
            "subsection_title": subsection_title,
            "numbers_in_narrative": [],
            "numbers_found_in_processed_data": [],
            "numbers_missing_in_processed_data": [],
            "count_numbers": 0,
            "count_found": 0,
            "count_missing": 0,
            "consistency": "N/A (no numbers in narrative)",
        })
        continue

    # --- enhanced check ---
    found, missing = check_number_consistency(numbers, processed)

    evaluation_results.append({
        "section_index": idx + 1,
        "section_title": section_title,
        "subsection_title": subsection_title,
        "numbers_in_narrative": numbers,
        "numbers_found_in_processed_data": found,
        "numbers_missing_in_processed_data": missing,
        "count_numbers": len(numbers),
        "count_found": len(found),
        "count_missing": len(missing),
        "consistency": "PASS" if len(missing) == 0 else "FAIL",
    })


# ============================================================
# 4. Print per-section results
# ============================================================

print("\n===== Evaluation Report =====\n")
for r in evaluation_results:
    print(f"--- Section {r['section_index']} ---")
    if r.get("section_title"):
        print("Section Title   :", r["section_title"])
    if r.get("subsection_title"):
        print("Subsection Title:", r["subsection_title"])

    print("Numbers in narrative       :", r["numbers_in_narrative"])
    print("Found in processed data    :", r["numbers_found_in_processed_data"])
    print("Missing in processed data  :", r["numbers_missing_in_processed_data"])
    print("Consistency                :", r["consistency"])
    print("Count (total / found / missing):",
          r["count_numbers"], "/", r["count_found"], "/", r["count_missing"])
    print("")


# ============================================================
# 5. Global coverage statistics
# ============================================================

total_numbers = sum(r["count_numbers"] for r in evaluation_results)
total_found = sum(r["count_found"] for r in evaluation_results)
total_missing = sum(r["count_missing"] for r in evaluation_results)

coverage = (total_found / total_numbers) * 100.0 if total_numbers else 0.0

print("===== Overall Coverage Statistics =====")
print(f"Total numeric mentions in narratives : {total_numbers}")
print(f"Total numbers found in analysis data : {total_found}")
print(f"Total numbers missing                : {total_missing}")
print(f"Overall coverage                     : {coverage:.2f}%")



===== Evaluation Report =====

--- Section 1 ---
Section Title   : Annual Performance and Market Structure Dynamics
Subsection Title: Comprehensive Annual Sales Volume and Revenue Trend
Numbers in narrative       : [2022.0, 2023.0, 7.7, 7.2, 1.8, 1.7, 0.1, 2022.0, 2023.0]
Found in processed data    : [2022.0, 2023.0, 7.7, 7.2, 1.8, 1.7, 0.1, 2022.0, 2023.0]
Missing in processed data  : []
Consistency                : PASS
Count (total / found / missing): 9 / 9 / 0

--- Section 2 ---
Section Title   : Annual Performance and Market Structure Dynamics
Subsection Title: Annual Shift in Fuel Type Market Preference
Numbers in narrative       : [3.6, 1.8, 2020.0, 2024.0, 2024.0, 26.5, 24.7, 2020.0, 2.0, 24.5, 2024.0, 1.2, 24.9, 0.4, 24.2, 100.0, 2024.0, 1.8, 2020.0, 2.4, 2024.0]
Found in processed data    : [3.6, 1.8, 2020.0, 2024.0, 2024.0, 26.5, 24.7, 2020.0, 2.0, 24.5, 2024.0, 1.2, 24.9, 0.4, 24.2, 2024.0, 1.8, 2020.0, 2024.0]
Missing in processed data  : [100.0, 2.4]
Consistency         